In [1]:
import sqlite3
import pprint as pp
import numpy as np
from sklearn.neighbors import NearestNeighbors
import datetime

In [2]:
def readTags(path, query):  
    conn = sqlite3.connect(path)
    res = conn.execute(query)
    tags = res.fetchall()
    # pp.pprint(data)
    print "Total: ", len(tags)
    return conn, tags

In [3]:
def createTermsIndices(tags):
    tagsDict = {}
    for index in range(len(tags)):
        tagsDict[tags[index][0]] = index
    print len(tagsDict.values())
    return tagsDict

In [4]:
def getArtistTerms(artist):
    query="SELECT term FROM artist_term where artist_id='%s';" %(artist)
    res = term.execute(query)
    artistTerms = res.fetchall()
    return artistTerms

In [5]:
def Timer():
    return datetime.datetime.now()

In [6]:
def createMatrix(artists, terms):
    mat = np.zeros((len(artists),len(terms)))
    print mat.shape

    for artistIndex in range(len(artists)):
        if artistIndex%10000 == 0:
            print "Processed %d items" %(artistIndex)
        artistTerms = getArtistTerms(artists[artistIndex])
        for artistTermIndex in range(len(artistTerms)):
            if artistTerms[artistTermIndex][0] in termsDict.keys():
    #             print termsDict[artistTerms[artistTermIndex][0]], artistTerms[artistTermIndex][0]
                mat[artistIndex][(termsDict[artistTerms[artistTermIndex][0]])]=1.0
    return mat

In [7]:
def createTable(filename, tablename):
    createConn = sqlite3.connect(filename)
    createTableQuery = """ CREATE TABLE IF NOT EXISTS %s (
                                        target text,
                                        similar text
                                    ); """ %(tablename)

    createConn.execute(createTableQuery)
    return createConn

In [8]:
def insertTable(createConn, tablename, indices, knn):
    counter = 0
    for currentIndex in indices:
        counter = counter+1
        if counter%10000 == 0:
            print "Inserted %d items" %(counter)
        for index in range(1, len(currentIndex)):
            if index == knn:
                break
    #         print artists[currentIndex[0]][0], artists[currentIndex[index]][0]
            insertQuery = """ INSERT INTO %s VALUES ('%s', '%s'); """ %(tablename, artists[currentIndex[0]][0], artists[currentIndex[index]][0])
    #         print insertQuery
            createConn.execute(insertQuery)
            

In [9]:
def checkTable(createConn, tablename):
    res =  createConn.execute("Select count(*) from %s;" %(tablename))
    print res.fetchall()

def deleteTable(createConn):
    createConn.execute("drop table %s;" %(tablename))

In [10]:
term, artists = readTags('data/artist_term.db', 'SELECT artist_id FROM artists;')
term, terms = readTags('data/artist_term.db',
                        'SELECT term FROM artist_term GROUP BY term HAVING count(*)>500;')
termsDict = createTermsIndices(terms)
mat = createMatrix(artists, terms)

Total:  44745
Total:  353
353
(44745, 353)
Processed 0 items
Processed 10000 items
Processed 20000 items
Processed 30000 items
Processed 40000 items


In [11]:
start = Timer()
print start
nbrs = NearestNeighbors(n_neighbors=200, algorithm='brute', metric='euclidean').fit(mat)
distances, indices = nbrs.kneighbors(mat)
print indices
print distances
end = Timer()
print end - start

2017-11-27 02:31:25.851203
[[    0 34301 19676 ..., 16367 40825 24162]
 [    1   294 11189 ..., 22887  3949  1313]
 [    2 32635  6864 ..., 20764 24286 36992]
 ..., 
 [44742 33447 25162 ..., 29041  1164 15548]
 [44060 42603 18464 ..., 34572 43902 41323]
 [44744 41754 36796 ..., 20473 23269 23238]]
[[ 0.          3.87298335  3.87298335 ...,  4.58257569  4.58257569
   4.69041576]
 [ 0.          2.          2.         ...,  2.64575131  2.64575131
   2.64575131]
 [ 0.          5.          5.38516481 ...,  6.32455532  6.32455532
   6.32455532]
 ..., 
 [ 0.          2.82842712  2.82842712 ...,  3.46410162  3.46410162
   3.46410162]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          3.          3.         ...,  3.74165739  3.74165739
   3.74165739]]
0:42:19.182163


## Calculating Accuracy

In [28]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'euclidean_tags_60')
checkTable(createConn, 'euclidean_tags_60')
insertTable(createConn, 'euclidean_tags_60', indices, 60)
checkTable(createConn, 'euclidean_tags_60')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(2639955,)]
0:00:50.840673


In [29]:
start = Timer()
learning_factor=7.3
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM euclidean_tags_60 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.756400
0:00:01.502975
0:00:02.248185
0:00:02.955314
0:00:03.653072
0:00:04.353280
0:00:05.042911
0:00:05.797383
0:00:06.499465
0:00:07.213265
0:00:07.914568
0:00:08.597734
0:00:09.275567
0:00:09.969112
0:00:10.666219
0:00:11.371575
0:00:12.133531
0:00:12.930057
0:00:13.718540
0:00:14.506827


In [30]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
print denominator

141.0


In [31]:
print "Accuracy Score for Euclidean for 60 Neighbours", total/denominator
createConn.close()

Accuracy Score for Euclidean for 60 Neighbours 0.687943262411


In [32]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'euclidean_tags_45')
checkTable(createConn, 'euclidean_tags_45')
insertTable(createConn, 'euclidean_tags_45', indices, 45)
checkTable(createConn, 'euclidean_tags_45')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(1968780,)]
0:00:39.381569


In [33]:
start = Timer()
total = 0.00
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM euclidean_tags_45 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:00.516283
0:00:01.034116
0:00:01.541837
0:00:02.041626
0:00:02.548986
0:00:03.065743
0:00:03.686193
0:00:04.199576
0:00:04.721202
0:00:05.247028
0:00:05.738452
0:00:06.244737
0:00:06.740321
0:00:07.252481
0:00:07.790338
0:00:08.425530
0:00:09.009104
0:00:09.572472
0:00:10.119183
0:00:10.637623


In [34]:
print "Accuracy Score for Euclidean for 45 Neighbours", total/denominator #45 neighbours algorithm: Ball_tree
createConn.close()

Accuracy Score for Euclidean for 45 Neighbours 0.58865248227


In [35]:
start = Timer()
createConn = createTable('data/artist_similarity.db', 'euclidean_tags_90')
checkTable(createConn, 'euclidean_tags_90')
insertTable(createConn, 'euclidean_tags_90', indices, 90)
checkTable(createConn, 'euclidean_tags_90')
end = Timer()
print end - start

[(0,)]
Inserted 10000 items
Inserted 20000 items
Inserted 30000 items
Inserted 40000 items
[(3982305,)]
0:01:13.891296


In [36]:
start = Timer()
total = 0.00
learning_factor=5.2
for i in range(0,20):
#     print artists[i][0]
    res = createConn.execute("SELECT * FROM euclidean_tags_90 as actual INNER JOIN similarity as predict ON actual.target = predict.target and actual.similar = predict.similar where actual.target='%s';" %(artists[i][0]))
    final = res.fetchall()
#     print len(final)
    total = total + len(final)
    end = Timer()
    print end - start

0:00:01.055694
0:00:02.151588
0:00:03.226044
0:00:04.262756
0:00:05.278198
0:00:06.314237
0:00:07.354676
0:00:08.373546
0:00:09.394024
0:00:10.401689
0:00:11.414701
0:00:12.434797
0:00:13.466739
0:00:14.477437
0:00:15.540047
0:00:16.589631
0:00:17.582066
0:00:18.593540
0:00:19.615412
0:00:20.632295


In [37]:
res = createConn.execute("SELECT count(*) FROM similarity GROUP BY target LIMIT 20;")
counts = res.fetchall()
denominator=0.00
for i in counts:
    denominator = denominator + int(i[0]/learning_factor)
# print denominator

In [38]:
print "Accuracy Score for Euclidean for 90 Neighbours", total/denominator #90 neighbours algorithm: Ball_tree
createConn.close()

Accuracy Score for Euclidean for 90 Neighbours 0.597014925373
